# Hands-On 1: Portable Parallel Programming with OpenMP

Welcome to Hands-on _Portable Parallel Programming with OpenMP_. This notebook comprises 2 sessions. Next table shows the documents and files needed to develop each one of the exercises.


|  Sessions     | Codes             | files           | 
| --------------| ----------------- | --------------- |
| Session 1     | Matrix Multiply   |  mm.c           |  
| Session 2     | Asynchronous Task |  asyncTaskOpenMP.c |   


## `Matrix Multiple Benchmark`

The definite algebraic operation of the matrix can be defined as:For your work today, you have access to several GPUs in the cloud. Run the following cell to see the GPUs available to you today.

$$ c_{ij} = \sum\limits_{k=1}^{n } a_{ik} b_{kj} = a_{i1}b_{1j} + a_{i2}b_{2j} + ... + a_{in}b_{nj} $$

where $i$ is summed over for all possible values of $j$ and $k$ and the notation above uses the summation convention. The sequential code of the program is available in the file `mm.c`. The follow code shows an extract of such code. In particular, we can see the algebraic operation include a loop that implements the summatory of the above definition.

In [ ]:
%%writefile mm.c
#include <stdio.h>
#include <stdlib.h>

void initializeMatrix(int *matrix, int size)
{
  for(int i = 0; i < size; i++)
    for(int j = 0; j < size; j++)
      matrix[i * size + j] = rand() % (10 - 1) * 1;
}

void printMatrix(int *matrix, int size)
{
  for(int i = 0; i < size; i++)
  {
    for(int j = 0; j < size; j++)
      printf("%d\t", matrix[i * size + j]);
    printf("\n");
  }
  printf("\n");
}

int main(int argc, char **argv)
{
  int size = atoi(argv[1]);  
  int i, j, k;

  int  *A = (int *) malloc (sizeof(int)*size*size);
  int  *B = (int *) malloc (sizeof(int)*size*size);
  int  *C = (int *) malloc (sizeof(int)*size*size);

  initializeMatrix(A, size);
  initializeMatrix(B, size);

  for(i = 0; i < size; i++)
   for(j = 0; j < size; j++)
     for(k = 0; k < size; k++)
        C[i * size + j] += A[i * size + k] * B[k * size + j];

  printMatrix(A,size);
  printMatrix(B,size);
  printMatrix(C,size);

  return 0;
}

### Run the Code

In [ ]:
!gcc mm.c -o mm

In [ ]:
!./mm 10

### Entering time measurement metrics

The next step will be to modify the code in the file `mm.c` to enter time measurement metrics of the matrix multiply in parallel using OpenMP. A first approach could be to use the command `omp_get_wtime()` for the get the initial and final time. You will need to link the command with the OpenMP library by including `omp.h`.

In [ ]:
%%writefile mm.c
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

void initializeMatrix(int *matrix, int size)
{
  for(int i = 0; i < size; i++)
    for(int j = 0; j < size; j++)
      matrix[i * size + j] = rand() % (10 - 1) * 1;
}

void printMatrix(int *matrix, int size)
{
  for(int i = 0; i < size; i++)
  {
    for(int j = 0; j < size; j++)
      printf("%d\t", matrix[i * size + j]);
    printf("\n");
  }
  printf("\n");
}

int main(int argc, char **argv)
{
 int size = atoi(argv[1]);  
 int i, j, k;
 double t1, t2;

 int  *A = (int *) malloc (sizeof(int)*size*size);
 int  *B = (int *) malloc (sizeof(int)*size*size);
 int  *C = (int *) malloc (sizeof(int)*size*size);

 initializeMatrix(A, size);
 initializeMatrix(B, size);

 t1 = omp_get_wtime();
   for(i = 0; i < size; i++)
    for(j = 0; j < size; j++)
      for(k = 0; k < size; k++)
        C[i * size + j] += A[i * size + k] * B[k * size + j];
 t2 = omp_get_wtime();

 printf("%d\t%f\n",size, t2-t1);

 //printMatrix(A,size);
 //printMatrix(B,size);
 //printMatrix(C,size);

 return 0;
}

### Run the Code

In [ ]:
!gcc mm.c -o mm -fopenmp

In [ ]:
!./mm 1000

### Inserting the OpenMP directive

The next step will be to modify the code in the file `mm.c` to perform the computation of the integral in parallel using OpenMP. A first approach could be to use the directive parallel for considering the variables $i$, $j$ and $k$ are private. After this change, you can compile and execute the program.

In [ ]:
%%writefile mm.c
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

void initializeMatrix(int *matrix, int size)
{
  for(int i = 0; i < size; i++)
    for(int j = 0; j < size; j++)
      matrix[i * size + j] = rand() % (10 - 1) * 1;
}

void printMatrix(int *matrix, int size)
{
  for(int i = 0; i < size; i++)
  {
    for(int j = 0; j < size; j++)
      printf("%d\t", matrix[i * size + j]);
    printf("\n");
  }
  printf("\n");
}

int main (int argc, char **argv)
{
 int size = atoi(argv[1]);  
 int i, j, k;
 double t1, t2;

 int  *A = (int *) malloc (sizeof(int)*size*size);
 int  *B = (int *) malloc (sizeof(int)*size*size);
 int  *C = (int *) malloc (sizeof(int)*size*size);

 initializeMatrix(A, size);
 initializeMatrix(B, size);

 t1 = omp_get_wtime();
 #pragma omp parallel for private(i, j, k)
   for(i = 0; i < size; i++)
    for(j = 0; j < size; j++)
      for(k = 0; k < size; k++)
        C[i * size + j] += A[i * size + k] * B[k * size + j];
 t2 = omp_get_wtime();

 printf("%d\t%f\n",size, t2-t1);

 //printMatrix(A,size);
 //printMatrix(B,size);
 //printMatrix(C,size);

 return 0;
}

### Run the Code

In [ ]:
!gcc mm.c -o mm -fopenmp

In [ ]:
!OMP_NUM_THREADS=16 ./mm 1000

### Performance Analysis

The last step will be to create the shell script file to measure the perform the computation of the matrix multiply in parallel using OpenMP. The shell script is available in the file `script.sh`.
Compile and execute the script. At run time, an argument can be used to select the number of the threads. For example, to use the first variant you can use for $16$ threads:

In [ ]:
%%writefile script.sh
#!/bin/sh

for i in 100 200 300 400 500 600 700 800 900 1000
do
  OMP_NUM_THREADS=$1  ./mm  "$i"
done

In [ ]:
!bash script.sh 16

## `Asynchronous Task`

Asynchronous programming is a set of techniques for implementing expensive operations that run concurrently with the rest of the program. One domain where asynchronous programming is often used is in programs with a graphical user interface: it is often unacceptable when the user interface freezes while performing a costly operation. Also, asynchronous operations are essential for parallel applications that need to run multiple tasks simultaneously. The following is a code `asyncTaskOpenMP.c` that represents a task being done asynchronously. Before understanding the code, compile and run it as follows:

In [ ]:
%%writefile asyncTaskOpenMP.c
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

#define SIZE_MATRIX 10

int main(int argc, char **argv)
{
  int n = atoi(argv[1]);
  int block_size = atoi(argv[2]);
  int matrix[SIZE_MATRIX][SIZE_MATRIX], k1 = 10, k2 = 20;
  int i, j, row, column;

  for(i = 0; i < n; i++)
  {
    for(j = 0; j < n; j++)
    {
      matrix[i][j] = 5;
      printf("%d\t", matrix[i][j]);
    }
    printf("\n");
  }

  printf("\n\n");

  omp_set_num_threads(5);

  #pragma omp parallel private(row, column)
  {
    int id = omp_get_thread_num();

    if(id == 0)
    {
      for(row = 0; row < n; row++)
        for(column = 0; column < block_size; column++)
          matrix[row][column] *= k1;
    }

    if(id == 1)
    {
      for(row = 0; row < n; row++)
        for(column = block_size; column < 2 * block_size; column++)
          matrix[row][column] *= k2;
    }
  
  }

  for(i = 0; i < n; i++)
  {
    for(j = 0; j < n; j++)
      printf("%d\t", matrix[i][j]);
    printf("\n");
  }

  return 0;
}

### Run the Code

In [ ]:
!gcc asyncTaskOpenMP.c -o asyncTaskOpenMP -fopenmp

In [ ]:
!./asyncTaskOpenMP 10 2

## References

M. Boratto. Hands-On Supercomputing with Parallel Computing. Available: https://github.com/muriloboratto/Hands-On-Supercomputing-with-Parallel-Computing. 2022.

B. Chapman, G. Jost and R. Pas. Using OpenMP: Portable Shared Memory Parallel Programming. The MIT Press, 2007, USA.